## Overview

[Agent Engine](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/overview) is a managed service that helps you to build and deploy agent frameworks. [LangGraph](https://langchain-ai.github.io/langgraph/) is a library for building stateful, multi-actor applications with LLMs, used to create agent and multi-agent workflows.

This notebook demonstrates how to build, deploy, and test a simple LangGraph application using [Agent Engine](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/overview) in Vertex AI. You'll learn how to combine LangGraph's workflow orchestration with the scalability of Vertex AI, which enables you to build custom generative AI applications.

Note that the approach used in this notebook defines a [custom application template in Agent Engine](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/customize), which can be extended to LangChain or other orchestration frameworks. If just want to use Agent Engine to build agentic generative AI applications, refer to the documentation for [developing with the LangChain template in Agent Engine](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/develop/overview).

This notebook covers the following steps:

- **Define Tools**: Create custom Python functions to act as tools your AI application can use.
- **Define Router**: Set up routing logic to control conversation flow and tool selection.
- **Build a LangGraph Application**: Structure your application using LangGraph, including the Gemini model and custom tools that you define.
- **Local Testing**: Test your LangGraph application locally to ensure functionality.
- **Deploying to Vertex AI**: Seamlessly deploy your LangGraph application to Agent Engine for scalable execution.
- **Remote Testing**: Interact with your deployed application through Vertex AI, testing its functionality in a production-like environment.
- **Cleaning Up Resources**: Delete your deployed application on Vertex AI to avoid incurring unnecessary charges.

By the end of this notebook, you'll have the skills and knowledge to build and deploy your own custom generative AI applications using LangGraph, Agent Engine, and Vertex AI.

## Get started

### Install Vertex AI SDK and other required packages

In [1]:
%pip install --upgrade --user --quiet \
    "google-cloud-aiplatform[agent_engines,langchain]" \
    cloudpickle==3.0.0 \
    "pydantic==2.11.2" \
    langgraph \
    httpx

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aider-chat 0.84.0 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.
aider-chat 0.84.0 requires google-auth==2.40.1, but you have google-auth 2.40.3 which is incompatible.
aider-chat 0.84.0 requires openai==1.75.0, but you have openai 1.86.0 which is incompatible.
aider-chat 0.84.0 requires pydantic==2.11.4, but you have pydantic 2.11.2 which is incompatible.
aider-chat 0.84.0 requires pydantic-core==2.33.2, but you have pydantic-core 2.33.1 which is incompatible.
aider-chat 0.84.0 requires requests==2.32.3, but you have requests 2.32.4 which is incompatible.
chromadb 0.5.23 requires tokenizers<=0.20.3,>=0.13.2, but you have tokenizers 0.21.1 which is incompatible.
crewai 0.121.0 requires litellm==1.68.0, but you have litellm 1.68.1 which is incom

### Restart runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which restarts the current kernel.

The restart might take a minute or longer. After it's restarted, continue to the next step.

In [2]:
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Wait until it's finished before continuing to the next step. ⚠️</b>
</div>

### Authenticate your notebook environment (Colab only)

If you're running this notebook on Google Colab, run the cell below to authenticate your environment.

In [1]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [2]:
LOCATION = "us-central1"  # @param {type:"string"}
STAGING_BUCKET = "gs://veretxai_image_recognition_02"  # @param {type:"string"}

PROJECT_ID="my-project-0004-346516"

if not PROJECT_ID:
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=STAGING_BUCKET)

## Building and deploying a LangGraph app on Agent Engine

In the following sections, we'll walk through the process of building and deploying a LangGraph application using Agent Engine in Vertex AI.

### Import libraries

Import the necessary Python libraries. These libraries provide the tools we need to interact with LangGraph, Vertex AI, and other components of our application.

In [3]:
from typing import Literal

from langchain_core.messages import BaseMessage, HumanMessage
from langchain_google_vertexai import ChatVertexAI
from langgraph.graph import END, MessageGraph
from langgraph.prebuilt import ToolNode
from vertexai import agent_engines

### Define tools

You'll start by defining the a tool for your LangGraph application. You'll define a custom Python function that act as tools in our agentic application.

In this case, we'll define a simple tool that returns a product description based on the product that the user asks about. In reality, you can write functions to call APIs, query databases, or anything other tasks that you might want your agent to be able to use.

In [4]:
def get_product_details(product_name: str):
    """Gathers basic details about a product."""
    details = {
        "smartphone": "A cutting-edge smartphone with advanced camera features and lightning-fast processing.",
        "coffee": "A rich, aromatic blend of ethically sourced coffee beans.",
        "shoes": "High-performance running shoes designed for comfort, support, and speed.",
        "headphones": "Wireless headphones with advanced noise cancellation technology for immersive audio.",
        "speaker": "A voice-controlled smart speaker that plays music, sets alarms, and controls smart home devices.",
    }
    return details.get(product_name, "Product details not found.")

### Define router

Then, you'll define a router to control the flow of the conversation, determining which tool to use based on user input or the state of the interaction. Here we'll use a simple router setup, and you can customize the behavior of your router to handle multiple tools, custom logic, or multi-agent workflows.

In [5]:
def router(state: list[BaseMessage]) -> Literal["get_product_details", "__end__"]:
    """Initiates product details retrieval if the user asks for a product."""
    # Get the tool_calls from the last message in the conversation history.
    tool_calls = state[-1].tool_calls
    # If there are any tool_calls
    if len(tool_calls):
        # Return the name of the tool to be called
        return "get_product_details"
    else:
        # End the conversation flow.
        return "__end__"

### Define LangGraph application

Now you'll bring everything together to define your LangGraph application as a custom template in Agent Engine.

This application will use the tool and router that you just defined. LangGraph provides a powerful way to structure these interactions and leverage the capabilities of LLMs.

In [6]:
class SimpleLangGraphApp:
    def __init__(self, project: str, location: str) -> None:
        self.project_id = project
        self.location = location

    # The set_up method is used to define application initialization logic
    def set_up(self) -> None:
        model = ChatVertexAI(model="gemini-2.0-flash")

        builder = MessageGraph()

        model_with_tools = model.bind_tools([get_product_details])
        builder.add_node("tools", model_with_tools)

        tool_node = ToolNode([get_product_details])
        builder.add_node("get_product_details", tool_node)
        builder.add_edge("get_product_details", END)

        builder.set_entry_point("tools")
        builder.add_conditional_edges("tools", router)

        self.runnable = builder.compile()

    # The query method will be used to send inputs to the agent
    def query(self, message: str):
        """Query the application.

        Args:
            message: The user message.

        Returns:
            str: The LLM response.
        """
        chat_history = self.runnable.invoke(HumanMessage(message))

        return chat_history[-1].content

### Local testing

In this section, you'll test your LangGraph app locally before deploying it to ensure that it behaves as expected before deployment.

In [7]:
agent = SimpleLangGraphApp(project=PROJECT_ID, location=LOCATION)
agent.set_up()

In [8]:
agent.query(message="Get product details for shoes")

'High-performance running shoes designed for comfort, support, and speed.'

In [9]:
agent.query(message="Get product details for coffee")

'A rich, aromatic blend of ethically sourced coffee beans.'

In [10]:
agent.query(message="Get product details for smartphone")

'A cutting-edge smartphone with advanced camera features and lightning-fast processing.'

In [11]:
# Ask a question that cannot be answered using the defined tools
agent.query(message="Tell me about the weather")

'I am sorry, I cannot provide weather information. I can only provide details about products.'

### Deploy your LangGraph app

Now that you verified that your LangGraph application is working locally, it's time to deploy it to Agent Engine! This will make your application accessible remotely and allow you to integrate it into larger systems or provide it as a service.

In [12]:
remote_agent = agent_engines.create(
    SimpleLangGraphApp(project=PROJECT_ID, location=LOCATION),
    requirements=[
        "google-cloud-aiplatform[agent_engines,langchain]",
        "cloudpickle==3.0.0",
        "pydantic==2.11.2",
        "langgraph",
        "httpx",
    ],
    display_name="Agent Engine with LangGraph",
    description="This is a sample custom application in Agent Engine that uses LangGraph",
    extra_packages=[],
)

Identified the following requirements: {'pydantic': '2.11.2', 'cloudpickle': '3.0.0'}
The final list of requirements: ['google-cloud-aiplatform[agent_engines,langchain]', 'cloudpickle==3.0.0', 'pydantic==2.11.2', 'langgraph', 'httpx']
Using bucket veretxai_image_recognition_02
Wrote to gs://veretxai_image_recognition_02/agent_engine/agent_engine.pkl
Writing to gs://veretxai_image_recognition_02/agent_engine/requirements.txt
Creating in-memory tarfile of extra_packages
Writing to gs://veretxai_image_recognition_02/agent_engine/dependencies.tar.gz
Creating AgentEngine
Create AgentEngine backing LRO: projects/255766800726/locations/us-central1/reasoningEngines/7257383673740132352/operations/1459027569004249088
View progress and logs at https://console.cloud.google.com/logs/query?project=my-project-0004-346516
AgentEngine created. Resource name: projects/255766800726/locations/us-central1/reasoningEngines/7257383673740132352
To use this AgentEngine in another session:
agent_engine = vertex

### Remote test

Now that your LangGraph app is running on Agent Engine, let's test it out by querying it in the remote environment:

In [13]:
remote_agent.query(message="Get product details for shoes")

'High-performance running shoes designed for comfort, support, and speed.'

In [14]:
remote_agent.query(message="Get product details for coffee")

'A rich, aromatic blend of ethically sourced coffee beans.'

In [15]:
remote_agent.query(message="Get product details for smartphone")

'A cutting-edge smartphone with advanced camera features and lightning-fast processing.'

In [16]:
remote_agent.query(message="Tell me about the weather")

'I am sorry, I cannot provide weather information. I can only gather basic details about a product. What product are you interested in?\n'

## Cleaning up

After you've finished experimenting, it's a good practice to clean up your cloud resources. You can delete the deployed Agent Engine instance to avoid any unexpected charges on your Google Cloud account.

In [17]:
# remote_agent.delete()

Delete Agent Engine backing LRO: projects/255766800726/locations/us-central1/operations/4607254814768758784
Agent Engine deleted. Resource name: projects/255766800726/locations/us-central1/reasoningEngines/7257383673740132352
